In [16]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from tqdm import tqdm_notebook as tqdm
from bs4 import BeautifulSoup as bs
import requests as req

In [17]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\user\AppData\Local\Temp\ipykernel_23780\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


input을 사용해서 검색어를 입력할수있도록하고

format을 사용해서 검색어를 받아 오도로 할수 있다는거 까지만 알려드릴께요^^

In [18]:
driver.get('https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EC%B9%9C%ED%99%98%EA%B2%BD%20%EC%A0%9C%ED%92%88&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=46&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start=1')

##### 뉴스페이지 url 가져오기

In [19]:
soup = bs(driver.page_source,'html.parser')

In [20]:
search = soup.select('a.news_tit')

In [21]:
for i in search:
    print(i['href'])

http://www.newsis.com/view/?id=NISX20230212_0002189611&cID=13001&pID=13000
http://moneys.mt.co.kr/news/mwView.php?no=2023021014465934746
https://www.nocutnews.co.kr/news/5894072
http://www.enewstoday.co.kr/news/articleView.html?idxno=1637541
https://www.etoday.co.kr/news/view/2221215
https://www.news1.kr/articles/4950509
https://www.econovill.com/news/articleView.html?idxno=602370
http://www.edaily.co.kr/news/newspath.asp?newsid=01692486635510192
http://www.dailyimpact.co.kr/news/articleView.html?idxno=92698
http://www.inews24.com/view/1565275


In [22]:
# search1 = soup.find_all('a', class_='news_tit')
# for i in search1:
#     print('https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EA%B7%80%EB%86%8D+%EB%B6%88%ED%8E%B8%ED%95%9C+%EC%A0%90' + i['href'])

In [26]:
url_list = []
date_list= []
for i in range(1,301):
    page = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EC%B9%9C%ED%99%98%EA%B2%BD%20%EC%A0%9C%ED%92%88&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=46&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start={}1'.format(i-1)
    driver.get(page)
    time.sleep(1.5)

    soup = bs(driver.page_source,'html.parser')
    search = soup.select('a.news_tit')
    search1 = soup.select('div.info_group > span.info')
    for i in search:
        url_list.append(i['href'])
        date_list.append(search1)
len(url_list), len(date_list)

(3000, 3000)

In [10]:
# hrefs_list = []
# for i in range(1,10):
#     page = 'https://news.sbs.co.kr/news/newsflash.do?plink=NEW&cooper=SBSNEWSMAIN&pageIdx={}'.format(i)
#     hrefs = soup.select('a.news')        
#     for i in hrefs:
#         hrefs_list.append('https://news.sbs.co.kr/' + i['href'])
# len(hrefs_list)

In [27]:
url_list

['http://www.newsis.com/view/?id=NISX20230212_0002189611&cID=13001&pID=13000',
 'http://moneys.mt.co.kr/news/mwView.php?no=2023021014465934746',
 'https://www.nocutnews.co.kr/news/5894072',
 'http://www.enewstoday.co.kr/news/articleView.html?idxno=1637541',
 'https://www.etoday.co.kr/news/view/2221215',
 'https://www.news1.kr/articles/4950509',
 'https://www.econovill.com/news/articleView.html?idxno=602370',
 'http://www.edaily.co.kr/news/newspath.asp?newsid=01692486635510192',
 'http://www.dailyimpact.co.kr/news/articleView.html?idxno=92698',
 'http://www.inews24.com/view/1565275',
 'http://www.edaily.co.kr/news/newspath.asp?newsid=01692486635510192',
 'https://www.sedaily.com/NewsView/29LQ079PVF',
 'http://www.dailyimpact.co.kr/news/articleView.html?idxno=92698',
 'https://www.yna.co.kr/view/AKR20230209135200003?input=1195m',
 'http://www.newsis.com/view/?id=NISX20230210_0002188100&cID=13001&pID=13000',
 'http://www.inews24.com/view/1565275',
 'https://www.yeongnam.com/web/view.php?k

In [ ]:
newsdata = {'참조주소':url_list, '날짜':date_list} 
current_news = pd.DataFrame(newsdata)
current_news

In [28]:
# 수집 후 url list 중복 확인
real_list = set(url_list)
blog_url_list = list(real_list)
len(blog_url_list)

2370

In [29]:
# 상세 페이지 기사 제목 & 내용
title_list = []
contents_list = []

for url in tqdm(url_list):
    driver.get(url)
    time.sleep(1)
    soup = bs(driver.page_source,'html.parser')
    
    try:
        title = soup.select('div.w_article_title > h3')
        contents = soup.select('div.text_area')
    
    except:
        title = soup.select('div.article > h1')
        contents = soup.select('div.content_area')
    
    for i in title:
        title_list.append(i.text)
        
    for i in contents:
        contents_list.append(i.text) 
len(title_list), len(contents_list)

C:\Users\user\AppData\Local\Temp\ipykernel_23780\957680578.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for url in tqdm(url_list):


  0%|          | 0/3000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
newsdata = {'제목':title_list,'날짜':date_list,'내용':contents_list,'참조주소':url_list} 
current_news = pd.DataFrame(newsdata)
current_news

,제목,내용
